<a href="https://colab.research.google.com/github/jm5159/E4511-2021-Mao/blob/master/chembl_curation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

%tensorflow_version 1.x

# get the Anaconda file 
! wget -c https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
! chmod +x Anaconda3-2019.10-Linux-x86_64.sh
! bash ./Anaconda3-2019.10-Linux-x86_64.sh -b -f -p /usr/local

! time conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import deepchem as dc

# install mordred, bravado and molvs
! time conda install -c conda-forge -y mordred bravado molvs

# get the Install AMPL_GPU_test.sh
!wget https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/config/install_AMPL_GPU_test.sh

# run the script to install AMPL
! chmod u+x install_AMPL_GPU_test.sh
! ./install_AMPL_GPU_test.sh

--2021-03-11 17:03:17--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88867207 (85M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.3-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  84.75M   257MB/s    in 0.3s    

2021-03-11 17:03:18 (257 MB/s) - ‘Miniconda3-py37_4.8.3-Linux-x86_64.sh’ saved [88867207/88867207]

PREFIX=/content/AMPL
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /content/AMPL

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - ca-certificates==2020.1.1=0
    - certifi==2020.4.5.1=py37_0
    - cffi==1.14.0=py37he30daa8_1
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.1=py37h7b6447c_0
    - conda==4.

In [ ]:
! wget https://raw.githubusercontent.com/jm5159/E4511-2021-Mao/master/Dataset/chembl.tsv

--2021-03-11 17:05:51--  https://raw.githubusercontent.com/jm5159/E4511_FNL_2021Spr/master/chembl.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5727350 (5.5M) [text/plain]
Saving to: ‘chembl.tsv’

chembl.tsv          100%[===================>]   5.46M  --.-KB/s    in 0.07s   

2021-03-11 17:05:52 (80.9 MB/s) - ‘chembl.tsv’ saved [5727350/5727350]



In [ ]:
# Load AMPL in this notebook

site_packages_path = '/content/AMPL/lib/python3.7/site-packages'
if site_packages_path not in sys.path:
  sys.path.insert(1, site_packages_path)
sys.path

['',
 '/content/AMPL/lib/python3.7/site-packages',
 '/content/AMPL/lib/python3.6/site-packages',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [ ]:
# There is a problem with the previously imported cffi, so delete it and load it with AMPL instead
if 'cffi' in sys.modules:
  del sys.modules['cffi']

In [ ]:
# manipulate data
import pandas as pd

# plot data
import numpy as np
import matplotlib.pyplot as plt

# curate data
import atomsci.ddm.utils.struct_utils as struct_utils
import atomsci.ddm.utils.curate_data as curate_data

# visualize compound structures
import tempfile
from rdkit import Chem
from rdkit.Chem import Draw
from itertools import islice
from IPython.display import Image, display

# visualize data
import seaborn as sns
sns.set_context('poster')
import matplotlib_venn as mpv
from scipy.stats import pearsonr

# set up visualization parameters
sns.set_context("poster")
sns.set_style("whitegrid")
sns.set_palette("Set2")
pal = sns.color_palette()
plt.rcParams['figure.figsize'] = [10,10]

pd.set_option('display.max_columns',(90))
pd.set_option('display.max_rows',(20))

DistributionNotFound: ignored

In [ ]:
dfp = pd.read_table("chembl.tsv")

In [ ]:

# Missing values for measurement column in this dataset
print("NA values:", dfp['Standard Value'].isna().sum())
dfp.drop(dfp[dfp['Standard Value'].isna()].index, inplace=True)

# canonicalize smiles strings
dfp['rdkit_smile'] = dfp['Smiles'].apply(curate_data.base_smiles_from_smiles)

#remove chemicals without smiles strings
print("NA SMILES:", (dfp.Smiles=='').sum()+dfp.Smiles.isna().sum())
dfp.drop(dfp[dfp.Smiles == ""].index, inplace=True)
dfp.drop(dfp[dfp.Smiles.isna()].index, inplace=True)
dfp.reset_index(drop=True, inplace=True)
dfp.head()

NA values: 1273
NA SMILES: 0


,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Data Validity Comment,Comment,Uo Units,Ligand Efficiency BEI,Ligand Efficiency LE,Ligand Efficiency LLE,Ligand Efficiency SEI,Potential Duplicate,Assay ChEMBL ID,Assay Description,Assay Type,BAO Format ID,BAO Label,Assay Organism,Assay Tissue ChEMBL ID,Assay Tissue Name,Assay Cell Type,Assay Subcellular Fraction,Target ChEMBL ID,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,rdkit_smile
0,CHEMBL240954,NaN,0,516.46,2,1.92,9,CC(=O)O[C@H]1[C@H](C)O[C@@H](Oc2c(-c3ccc(O)cc3...,IC50,'=',20600.0,nM,4.69,NaN,NaN,UO_0000065,NaN,NaN,NaN,NaN,False,CHEMBL1000805,Inhibition of human CYP3A4 by radiometric assay,A,BAO_0000357,single protein format,Homo sapiens,None,None,None,None,CHEMBL340,Cytochrome P450 3A4,Homo sapiens,SINGLE PROTEIN,CHEMBL1151241,1,Scientific Literature,J. Nat. Prod.,2004.0,None,CC(=O)O[C@H]1[C@H](C)O[C@@H](Oc2c(-c3ccc(O)cc3...
1,CHEMBL393363,NaN,0,528.54,1,3.67,29-rac,CCc1nc(N)nc(N)c1-c1ccc2c(c1)N(CCNC(C)=O)C(=O)C...,IC50,'=',120.0,nM,6.92,NaN,NaN,UO_0000065,NaN,NaN,NaN,NaN,False,CHEMBL898579,Inhibition of CYP3A4,A,BAO_0000357,single protein format,None,None,None,None,None,CHEMBL340,Cytochrome P450 3A4,Homo sapiens,SINGLE PROTEIN,CHEMBL1149625,1,Scientific Literature,Bioorg. Med. Chem.,2007.0,None,CCc1nc(N)nc(N)c1-c1ccc2c(c1)N(CCNC(C)=O)C(=O)C...
2,CHEMBL398584,NaN,0,496.52,0,2.93,26-rac,CCc1nc(N)nc(N)c1-c1ccc2c(c1)N(CCNC(C)=O)C(=O)C...,IC50,'=',860.0,nM,6.07,NaN,NaN,UO_0000065,NaN,NaN,NaN,NaN,False,CHEMBL898579,Inhibition of CYP3A4,A,BAO_0000357,single protein format,None,None,None,None,None,CHEMBL340,Cytochrome P450 3A4,Homo sapiens,SINGLE PROTEIN,CHEMBL1149625,1,Scientific Literature,Bioorg. Med. Chem.,2007.0,None,CCc1nc(N)nc(N)c1-c1ccc2c(c1)N(CCNC(C)=O)C(=O)C...
3,CHEMBL514681,NaN,0,406.91,0,4.29,8,O=C(CCc1ccccc1)NCC(=O)NC(c1ccccc1)c1ccc(Cl)cc1,IC50,'>',50000.0,nM,NaN,NaN,NaN,UO_0000065,NaN,NaN,NaN,NaN,False,CHEMBL995045,Inhibition of CYP3A4,A,BAO_0000357,single protein format,None,None,None,None,None,CHEMBL340,Cytochrome P450 3A4,Homo sapiens,SINGLE PROTEIN,CHEMBL1144246,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2008.0,None,O=C(CCc1ccccc1)NCC(=O)NC(c1ccccc1)c1ccc(Cl)cc1
4,CHEMBL452423,NaN,0,379.85,0,3.37,14,O=C(CNC(=O)c1ccncc1)NC(c1ccccc1)c1ccc(Cl)cc1,IC50,'>',50000.0,nM,NaN,NaN,NaN,UO_0000065,NaN,NaN,NaN,NaN,False,CHEMBL995045,Inhibition of CYP3A4,A,BAO_0000357,single protein format,None,None,None,None,None,CHEMBL340,Cytochrome P450 3A4,Homo sapiens,SINGLE PROTEIN,CHEMBL1144246,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2008.0,None,O=C(CNC(=O)c1ccncc1)NC(c1ccccc1)c1ccc(Cl)cc1


In [ ]:
print('Name duplicates:', dfp['Molecule ChEMBL ID'].duplicated().sum())
print('SMILES duplicates:', dfp.rdkit_smile.duplicated().sum())
print('Shape:',dfp.shape)

Name duplicates: 1539
SMILES duplicates: 1545
Shape: (8629, 42)


In [ ]:
# threshold -> pIC50
import numpy as np
thresh=-np.log10(1/1000000)
thresh

6.0

In [ ]:
dfp['Standard Value']=-np.log10(np.power(dfp['Standard Value'],-9))

In [ ]:
#average duplicated data
dfp_cur = curate_data.aggregate_assay_data(dfp, 
                         value_col='Standard Value', 
                         output_value_col='pIC50',
                         label_actives=True, 
                         active_thresh=thresh,
                         id_col='Molecule ChEMBL ID', 
                         smiles_col='rdkit_smile',
                         relation_col='Standard Relation', 
                         date_col=None)

0 entries in input table are missing SMILES strings
7084 unique SMILES strings are reduced to 7082 unique base SMILES strings


In [ ]:
dfp_cur.to_csv('CYP3A4_chembl_curated.csv', index=False)

In [ ]:
dfp_cur

,compound_id,base_rdkit_smiles,relation,pIC50,active
0,CHEMBL408443,Cc1cc2c(F)c(Oc3ncnn4cc(OC[C@@H](C)OC(=O)[C@H](...,,20.456664,1
1,CHEMBL2441953,CCCN(C)C(=O)CN1Cc2nc(C)c(CN)c(-c3ccc(Cl)cc3Cl)...,,33.663264,1
2,CHEMBL482088,COCCOC1CCN(c2cc(C)c3nc(-c4c(NC[C@@H](O)c5cccc(...,,-0.411817,0
3,CHEMBL3137442,Cc1ncsc1C(=O)N1CCC(N(c2ccc(C(F)(F)F)cc2)c2cccn...,,31.161582,1
4,CHEMBL3964219,CN[C@@H](C)C(=O)Nc1ccc(-c2c(-c3ccc(OC)nc3)nc3c...,,42.290730,1
...,...,...,...,...,...
7077,CHEMBL586213,COc1cc2c(cc1C(F)(F)F)N(C(=O)Nc1cc(OC(F)(F)F)cc...,,42.290730,1
7078,CHEMBL2057517,CNc1nc2ccc(C(=O)N(CC(C)C)C[C@@H](O)[C@H](Cc3cc...,,14.515055,1
7079,CHEMBL2382414,N#Cc1cnccc1COc1cnc(N2CCN(C(=O)OCC(F)(F)F)CC2)nc1,,40.294091,1
7080,CHEMBL2181890,COCC(=O)N[C@@H](Cc1cccc(-c2ncco2)c1)[C@H](O)CN...,,18.000000,1
